In [1]:
def sudoku(f):

    def af(g):
        for n, l in enumerate(g):
            for m, c in enumerate(l):
                P(str(c).replace("0", "."), end="")
                if m in {2, 5}:
                    P("+", end="")
            P()
            if n in {2, 5}:
                P("+" * 11)

    def cp(q, s):
        l = set(s[q[0]])
        l |= {s[i][q[1]] for i in range(9)}
        k = q[0] // 3, q[1] // 3
        for i in range(3):
            l |= set(s[k[0] * 3 + i][k[1] * 3:(k[1] + 1) * 3])
        return set(range(1, 10)) - l

    def ec(l):
        q = set(l) - {0}
        for c in q:
            if l.count(c) != 1:
                return True
        return False

    P = print
    af(f)

    s = []
    t = []
    for nl, l in enumerate(f):
        try:
            n = list(map(int, l))
        except:
            P("Line " + str(nl + 1) + " contains something other than a number.")
            return
        if len(n) != 9:
            P("Line " + str(nl + 1) + " does not contain 9 digits.")
            return
        t += [[nl, i] for i in range(9) if n[i] == 0]
        s.append(n)
    if nl != 8:
        P("The game contains " + str(nl + 1) + " lines instead of 9.")
        return

    for l in range(9):
        if ec(s[l]):
            P("Line " + str(l + 1) + " is contradictory.")
            return
    for c in range(9):
        k = [s[l][c] for l in range(9)]
        if ec(k):
            P("The column " + str(c + 1) + " is contradictory.")
            return
    for l in range(3):
        for c in range(3):
            q = []
            for i in range(3):
                q += s[l * 3 + i][c * 3:(c + 1) * 3]
            if ec(q):
                P("The cell (" + str(l + 1) + ";" +
                  str(c + 1) + ") is contradictory.")
                return

    p = [[] for i in t]
    cr = 0

    while cr < len(t):
        p[cr] = cp(t[cr], s)
        try:
            while not p[cr]:
                s[t[cr][0]][t[cr][1]] = 0
                cr -= 1
        except:
            P("Sudoku has no solution.")
            return
        s[t[cr][0]][t[cr][1]] = p[cr].pop()
        cr += 1

    af(s)
    return(s)


In [2]:
import cv2
import numpy as np
import operator
from keras.models import load_model
from keras.models import model_from_json
import sudoku_solver as sol

classifier = load_model("./digit_model.h5")

marge = 4
case = 28 + 2 * marge
taille_grille = 9 * case

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
flag = 0
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (1080, 620))


while True:

    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 9, 2)

    contours, hierarchy = cv2.findContours(
        thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_grille = None
    maxArea = 0

    for c in contours:
        area = cv2.contourArea(c)
        if area > 25000:
            peri = cv2.arcLength(c, True)
            polygone = cv2.approxPolyDP(c, 0.01 * peri, True)
            if area > maxArea and len(polygone) == 4:
                contour_grille = polygone
                maxArea = area

    if contour_grille is not None:
        cv2.drawContours(frame, [contour_grille], 0, (0, 255, 0), 2)
        points = np.vstack(contour_grille).squeeze()
        points = sorted(points, key=operator.itemgetter(1))
        if points[0][0] < points[1][0]:
            if points[3][0] < points[2][0]:
                pts1 = np.float32([points[0], points[1], points[3], points[2]])
            else:
                pts1 = np.float32([points[0], points[1], points[2], points[3]])
        else:
            if points[3][0] < points[2][0]:
                pts1 = np.float32([points[1], points[0], points[3], points[2]])
            else:
                pts1 = np.float32([points[1], points[0], points[2], points[3]])
        pts2 = np.float32([[0, 0], [taille_grille, 0], [0, taille_grille], [
                          taille_grille, taille_grille]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        grille = cv2.warpPerspective(frame, M, (taille_grille, taille_grille))
        grille = cv2.cvtColor(grille, cv2.COLOR_BGR2GRAY)
        grille = cv2.adaptiveThreshold(
            grille, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 7, 3)

        cv2.imshow("grille", grille)
        if flag == 0:

            grille_txt = []
            for y in range(9):
                ligne = ""
                for x in range(9):
                    y2min = y * case + marge
                    y2max = (y + 1) * case - marge
                    x2min = x * case + marge
                    x2max = (x + 1) * case - marge
                    cv2.imwrite("mat" + str(y) + str(x) + ".png",
                                grille[y2min:y2max, x2min:x2max])
                    img = grille[y2min:y2max, x2min:x2max]
                    x = img.reshape(1, 28, 28, 1)
                    if x.sum() > 10000:
                        prediction=np.argmax(classifier.predict(x), axis=-1)
                        ligne += "{:d}".format(prediction[0])
                    else:
                        ligne += "{:d}".format(0)
                grille_txt.append(ligne)
            print(grille_txt,end='\n')
            result = sol.sudoku(grille_txt)
        print("Resultant:", result)
        

        if result is not None:
            flag = 1
            fond = np.zeros(
                shape=(taille_grille, taille_grille, 3), dtype=np.float32)
            for y in range(len(result)):
                for x in range(len(result[y])):
                    if grille_txt[y][x] == "0":
                        cv2.putText(fond, "{:d}".format(result[y][x]), ((
                            x) * case + marge + 3, (y + 1) * case - marge - 3), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 0.9, (0, 0, 255), 1)
            M = cv2.getPerspectiveTransform(pts2, pts1)
            h, w, c = frame.shape
            fondP = cv2.warpPerspective(fond, M, (w, h))
            img2gray = cv2.cvtColor(fondP, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask = mask.astype('uint8')
            mask_inv = cv2.bitwise_not(mask)
            img1_bg = cv2.bitwise_and(frame, frame, mask=mask_inv)
            img2_fg = cv2.bitwise_and(fondP, fondP, mask=mask).astype('uint8')
            dst = cv2.add(img1_bg, img2_fg)
            dst = cv2.resize(dst, (1080, 620))
            cv2.imshow("frame", dst)
            out.write(dst)

        else:
            frame = cv2.resize(frame, (1080, 620))
            cv2.imshow("frame", frame)
            out.write(frame)

    else:
        flag = 0
        frame = cv2.resize(frame, (1080, 620))
        cv2.imshow("frame", frame)
        out.write(frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break


out.release()
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 16ms/step
['200007860', '000160000', '800009140', '008946700', '000003028', '070280910', '970032074', '003570009', '080600351']
2..+..7+86.
...+16.+...
8..+..9+14.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
97.+.32+.74
..3+57.+..9
.8.+6..+351
La ligne 7 est contradictoire.
Resultant: None
1/1 [==============================] - 0s 17ms/step
['200007860', '000160000', '800009140', '008946700', '000003028', '070280910', '910032074', '003570009', '080600351']
2..+..7+86.
...+16.+...
8..+..9+14.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
91.+.32+.74
..3+57.+..9
.8.+6..+351
291+457+863
437+168+592
856+329+147
+++++++++++
128+946+735
569+713+428
374+285+916
+++++++++++
915+832+674
643+571+289
782+694+351
Resultant: [[2, 9, 1, 4, 5, 7, 8, 6, 3], [4, 3, 7, 1, 6, 8, 5, 9, 2], [8, 5, 6, 3, 2, 9, 1, 4, 7], [1, 2, 8, 9, 4, 6, 7, 3, 5], [5, 6, 9, 7, 1, 3, 4, 2, 8], [3, 7, 4, 2, 8, 5, 9, 1, 6], [9, 1, 5, 8, 3, 2, 6, 7, 4], [6

1/1 [==============================] - 0s 22ms/step
['200007860', '000160000', '800009140', '008946700', '000003028', '070280910', '910032074', '003570009', '080600351']
2..+..7+86.
...+16.+...
8..+..9+14.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
91.+.32+.74
..3+57.+..9
.8.+6..+351
291+457+863
437+168+592
856+329+147
+++++++++++
128+946+735
569+713+428
374+285+916
+++++++++++
915+832+674
643+571+289
782+694+351
Resultant: [[2, 9, 1, 4, 5, 7, 8, 6, 3], [4, 3, 7, 1, 6, 8, 5, 9, 2], [8, 5, 6, 3, 2, 9, 1, 4, 7], [1, 2, 8, 9, 4, 6, 7, 3, 5], [5, 6, 9, 7, 1, 3, 4, 2, 8], [3, 7, 4, 2, 8, 5, 9, 1, 6], [9, 1, 5, 8, 3, 2, 6, 7, 4], [6, 4, 3, 5, 7, 1, 2, 8, 9], [7, 8, 2, 6, 9, 4, 3, 5, 1]]
1/1 [==============================] - 0s 17ms/step
['200007860', '000160000', '800009140', '008946700', '000003028', '070280910', '910032074', '003570009', '080600351']
2..+..7+86.
...+16.+...
8..+..9+14.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
91.+.32+.74
..3+57.+..9
.8.

1/1 [==============================] - 0s 16ms/step
['200007860', '000760000', '800009740', '008946700', '000003028', '070280910', '910032074', '003570009', '080600351']
2..+..7+86.
...+76.+...
8..+..9+74.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
91.+.32+.74
..3+57.+..9
.8.+6..+351
La colonne 7 est contradictoire.
Resultant: None
1/1 [==============================] - 0s 19ms/step
['200007860', '000160000', '800009140', '008946700', '000003028', '070280910', '930032074', '003570009', '080600351']
2..+..7+86.
...+16.+...
8..+..9+14.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
93.+.32+.74
..3+57.+..9
.8.+6..+351
La ligne 7 est contradictoire.
Resultant: None
1/1 [==============================] - 0s 17ms/step
['200007860', '000160000', '800009140', '008946700', '000003028', '070280910', '910032074', '003570009', '080600351']
2..+..7+86.
...+16.+...
8..+..9+14.
+++++++++++
..8+946+7..
...+..3+.28
.7.+28.+91.
+++++++++++
91.+.32+.74
..3+57.+..9
.8.+6..+35